In [1]:
from zipfile import ZipFile
file_name = "test_data/2022-ntust-information-retrieval-hw1.zip"
with ZipFile(file_name, 'r',) as zip:
    zip.extractall(path = 'test_data/extractHere/2022-ntust-information-retrieval-hw1')
    print('Done!!')
    All_lst = zip.namelist()

Query_lst = []
Doc_lst = []
for i in All_lst:
    if "documents/" in i:
        Doc_lst.append(i)
    elif "queries/" in i:
        Query_lst.append(i)

Done!!


In [2]:
# 可以在這建立lexicon
lexicon = {}
data_addr = 'test_data/extractHere/2022-ntust-information-retrieval-hw1/'

for i in Query_lst:
    file = open(data_addr + i, errors = 'ignore').read().split()
    for term in file:
        lexicon[term] = 0

# 要把document所有字也加進lexicon?
# for i in Doc_lst:
#     file = open("test_data/extractHere/" + i, errors = 'ignore').read().split()
#     for term in file:
#         lexicon[term] = 0

print(len(lexicon))
# print(lexicon)

190


In [3]:
import time

# 計算IDF中的n_i
Q_small_n = lexicon.copy()
D_small_n = lexicon.copy()
check = lexicon.copy()
# for term in check:
#     check[term] = False

start = time.time()
for q in Query_lst:
    file = open(data_addr + q, errors = 'ignore').read().split()
    for term in check:
        check[term] = False
    
    for term in file:
        if check[term] == False:
            Q_small_n[term] = Q_small_n[term] + 1 
            check[term] = True

for d in Doc_lst:
    file = open(data_addr + d, errors = 'ignore').read().split()
    for term in check:
        check[term] = False
        
    for term in file:
        if term in lexicon and check[term] == False:
            D_small_n[term] = D_small_n[term] + 1
            check[term] = True

end = time.time()
print("execution time(all small n): %f" %(end - start))

Q_small_n = list(Q_small_n.values())
D_small_n = list(D_small_n.values())

# print("Q_small_n: ", Q_small_n)
# print("D_small_n: ", D_small_n)

execution time(all small n): 3.963565


In [4]:
import math
import numpy as np


#每一迴圈計算一個document相對於所有lexicon的tf-idf，並存成一個list
start = time.time()
D_tfidf_lst = []

for i in range(len(Doc_lst)):
    D_k = lexicon.copy()
    D_tfidf = [0 for _  in range(len(lexicon))]
  
    file = open(data_addr + Doc_lst[i], errors = 'ignore').read().split()
    for term in file:
        if term in lexicon:
            D_k[term] = D_k[term] + 1

            
# document TF-IDF option, D_k = times, D_tfidf = weights
#     D_tfidf[j] = D_k[j] * math.log(len(Doc_lst) / D_small_n[j])
#     D_tfidf[j] = 1 + D_k[j]
#     D_tfidf[j] = (1 + D_k[j]) * math.log(len(Doc_lst) / D_small_n[j])
    D_k = list(D_k.values())
    D_tfidf = np.multiply(np.log10(np.ones(len(D_k)) + D_k), np.log10(np.divide(np.ones(len(D_k))*len(Doc_lst), np.ones(len(D_k)) +D_small_n))
)
  
    D_tfidf_lst.append(D_tfidf)
end = time.time()
print("execution time(all D_tfidf): %f" %(end - start))

execution time(all D_tfidf): 0.404155


In [5]:
import csv


start = time.time()
# 每一迴圈輸出一個Query的結果到test.csv
with open("test_data/test.csv", "w", newline='') as Answer_csv:
    writer = csv.writer(Answer_csv)
    writer.writerow(['Query', 'RetrievedDocuments'])
    
#     for item in Q_small_n:
#         if item < len(Query_lst):
#             item = item+1
    for ans in range(len(Query_lst)):
        Q_k = lexicon.copy()
        Q_tfidf = [0 for _ in range(len(lexicon))]
        sim_list = [0 for _ in range(len(Doc_lst))]

        file = open(data_addr + Query_lst[ans], errors = 'igonore').read().split()
        for term in file:
            if term in lexicon:
                Q_k[term] = Q_k[term] + 1

    
#     Query TF-IDF option, Q_k = times, Q_tfidf = weights
#         Q_tfidf[i] = (0.5 + 0.5*(Q_k[j]/max(Q_k))) * math.log(len(Query_lst) / Q_small_n[j])
#         Q_tfidf[i] = math.log(1 + len(Query_lst) / Q_small_n[i])
#         Q_tfidf[i] = (1 + Q_k[i]) * math.log(len(Query_lst) / Q_small_n[i])
#         if Q_small_n[term] < len(Query_lst):
#             Q_tfidf[term] = math.log(1 + Q_k[term]) * math.log(len(Query_lst) / (Q_small_n[term] + 1))
#         else:
#             Q_tfidf[term] = math.log(1 + Q_k[term]) * math.log(len(Query_lst) / (Q_small_n[term]))
#         Q_k = list(Q_k.values())
#         Q_tfidf = np.multiply(np.log10(np.ones(len(Q_k)) + Q_k), np.log2(np.divide(np.ones(len(Q_k))*len(Query_lst), Q_small_n)))
#         Q_tfidf = np.multiply(np.log(np.ones(len(Q_k)) + Q_k), np.log(np.divide(np.ones(len(Q_k))*len(Query_lst), Q_small_n+np.ones(len(lexicon)))))
        
        Q_k = list(Q_k.values())
        Q_k_temp = Q_k.copy()
        Q_k_temp = Q_k_temp + np.ones(len(Q_k))
        for i in range(len(Q_k_temp)):
            Q_k_temp[i] = math.log(Q_k_temp[i], 100)
        Q_tfidf = np.multiply((Q_k_temp), np.log10(np.divide(np.ones(len(Q_k))*len(Query_lst), Q_small_n)))
        
        
        for i in range(len(D_tfidf_lst)):
            sim_list[i] = np.sum(np.multiply(Q_tfidf, D_tfidf_lst[i])) / ( math.sqrt(np.sum(np.multiply(Q_tfidf, Q_tfidf))) * math.sqrt(np.sum(np.multiply(D_tfidf_lst[i], D_tfidf_lst[i])))+1)
    
        sorted_Doc = sorted(range(len(sim_list)), key=lambda k: sim_list[k], reverse=True)

        test_str = Doc_lst[sorted_Doc[0]].replace('documents/', '').replace(".txt", '')
        for i in range(1, len(Doc_lst)): 
            test_str = test_str + ' ' + Doc_lst[sorted_Doc[i]].replace('documents/', '').replace(".txt", '')
    
        writer.writerow([Query_lst[ans].replace('queries/', '').replace(".txt", '')] + [test_str])

end = time.time()
print("execution time(all Q_tfidf): %f" %(end - start))

execution time(all Q_tfidf): 0.825123
